## 3D-DAOSTORM 2D / 2D fixed analysis.
This notebook explains how to do 2D or 2D fixed analysis using 3D-DAOSTORM.
* In 2D fixed fitting we constrain the Gaussian fitting function with a fixed $\sigma$ value.
* In 2D fitting the Gaussian $\sigma$ can vary, but it is forced to be the same in X and Y.

### Configuring the directory
Create an empty directory somewhere on your computer and tell Python to go to that directory.

In [ ]:
import os
os.chdir("/home/hbabcock/Data/storm_analysis/jy_testing/")
print(os.getcwd())

Generate sample data for analysis.

In [ ]:
import storm_analysis.jupyter_examples.dao3d_2d as dao3d_2d
dao3d_2d.configure()

### Working with analysis parameters.
In this example we'll only adjust `threshold` but other important parameters include `sigma`, `roi_size` and `find_max_radius`.

In [ ]:
import storm_analysis.sa_library.parameters as params
daop = params.ParametersDAO().initFromFile("example.xml")

Getting help with a parameter:

In [ ]:
print(daop.helpAttr("threshold"))

Changing or getting a parameter. Here we are setting `max_frame` to 1 so that 3D-DAOSTORM will only analyze the first frame.

In [ ]:
daop.changeAttr("max_frame", 1)
print("max_frame is", daop.getAttr("max_frame"))

Print out all of the available parameters and their current values.

In [ ]:
daop.prettyPrint()

### Testing analysis parameters.

In [ ]:
import os
import storm_analysis.jupyter_examples.overlay_image as overlay_image
import storm_analysis.daostorm_3d.mufit_analysis as mfit


In [ ]:
# For this data-set, no localizations will be found if threshold is above 25.0
daop.changeAttr("threshold", 6.0)

daop.changeAttr("find_max_radius", 5) # original value is 5 (pixels)
daop.changeAttr("roi_size", 9) # original value is 9 (pixels)
daop.changeAttr("sigma", 1.5) # original value is 1.5 (pixels)

# Save the changed parameters.
#
# Using pretty = True will create a more human readable XML file. The default value is False.
#
daop.toXMLFile("testing.xml", pretty = True)

Test 3D-DAOSTORM analysis with these parameters. Ideally it should find 120 localizations in the frame.

Note that 3D-DAOSTORM will first check for existing analysis so you have to delete the old analysis after changing parameters.

In [ ]:
if os.path.exists("testing.hdf5"):
    os.remove("testing.hdf5")
    
mfit.analyze("test.tif", "testing.hdf5", "testing.xml")
overlay_image.overlayImage("test.tif", "testing.hdf5", 0)

### Using Visualizer

An alternative way to visualize the results is to use the visualizer program. This will only work if you are running jupyter locally.

In [ ]:
import inspect
import storm_analysis

vis_path = os.path.dirname(inspect.getfile(storm_analysis)) + "/visualizer/"

vis_cmd = vis_path + "/visualizer.py"
vis_dir = os.getcwd()

%run $vis_cmd $vis_dir

print(vis_path)

### Analyzing the whole movie

In [ ]:
# This tells 3D-DAOSTORM to analyze the whole movie.
daop.changeAttr("max_frame", -1)
daop.toXMLFile("final.xml")

# Delete any stale results.
if os.path.exists("final.hdf5"):
    os.remove("final.hdf5")
    
# Run the analysis.
mfit.analyze("test.tif", "final.hdf5", "final.xml")


### Creating an image from the analysis

In [ ]:
import matplotlib
import matplotlib.pyplot as pyplot

import storm_analysis.sa_utilities.hdf5_to_image as h5_image

sr_im = h5_image.render2DImage("final.hdf5", scale = 1, sigma = 1)

fig = pyplot.figure(figsize = (8, 8))
ax = fig.add_subplot(1,1,1)
ax.imshow(sr_im)
ax.set_title("SR Image")

pyplot.show()
